# Part 3: Feature Engineering

Using the Glassdoor data scientist jobs data set that I enriched, cleaned and reduced in Part 2, I will now extract features from the job titles and "raw" job descriptions, to prepare the data for analysis. 

## Setup

### Import Packages & Modules

In [1]:
# import packages and modules
import pandas as pd
import numpy as np
import os
import re
import matplotlib.pyplot as plt
import seaborn as sns
import random
import pickle


### Display Settings

In [2]:
# ensure all columns and rows will be displayed if/when you print the dataframe
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

# ensure all figures will have a white background in this notebook
%config InlineBackend.print_figure_kwargs={'facecolor' : "w"}

### Import Data

In [3]:
# provide the path to the location of the cleaned scraped glassdoor job data
path = './data/'

# provide glassdoor scrape date
scrapedate = '14Dec2020'  # e.g. '14Dec2020', '16Feb2021'

# create the absolute path to the cleaned glassdoor job data
filename = os.path.join(path, f"dsjobs_df_{scrapedate}_postclean.pkl")

# read the cleaned data scientist jobs data (.pkl file) into a dataframe
dsjobs = pd.read_pickle(filename)

# display dataframe info to check that it's what you expected
dsjobs.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 511 entries, 0 to 510
Data columns (total 33 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   job_title               511 non-null    object  
 1   salary_estimate         322 non-null    object  
 2   job_description         511 non-null    object  
 3   rating                  399 non-null    float64 
 4   company_name            511 non-null    object  
 5   location                511 non-null    object  
 6   size                    415 non-null    category
 7   founded                 348 non-null    Int64   
 8   type_of_ownership       428 non-null    category
 9   industry                381 non-null    category
 10  sector                  383 non-null    category
 11  revenue                 271 non-null    category
 12  rating_culturevalues    389 non-null    float64 
 13  rating_worklifebalance  394 non-null    float64 
 14  rating_diversity        29

## Seniority of position: Identify seniority from the job title

I am interested in being able to differentiate between junior and senior positions, so I'll now use the job titles to mark - using Boolean masks - those that mention seniority for the purpose of splitting and comparing these jobs later. 

In [4]:
# use a regular expression to identify junior positions from the job titles
dsjobs["seniority_junior"] = dsjobs["job_title"].str.contains(
    r"\bjunior\b|\bjr.?\b|\bentry level\b|\bgraduate\b", flags=re.IGNORECASE, regex=True)
dsjobs["seniority_junior"].value_counts().sort_index(ascending=False)


True      11
False    500
Name: seniority_junior, dtype: int64

There are very few job titles indicating a "junior role", so I will not analyse this group of jobs separately. I will remove the "seniority_junior" column. 

In [5]:
# drop the "seniority_junior" column
dsjobs.drop(columns="seniority_junior", inplace=True)

In [6]:
# use a regular expression to identify senior positions from the job titles
dsjobs["seniority_senior"] = dsjobs["job_title"].str.contains(
    r"\bsenior\b|\bsr.?\b|\bexperienced\b|\blead\b|\bprinciple\b|\bchief\b|\bmanager\b|\bhead\b", flags=re.IGNORECASE, regex=True)
dsjobs["seniority_senior"].value_counts().sort_index(ascending=False)


True     140
False    371
Name: seniority_senior, dtype: int64

A large proportion of the job titles in the dataset indicate being of a "senior" position, so this Boolean mask (or, new "feature") will be useful for analysing these roles separately. 

## Skills, experience & education: Job description text analysis

I want to know which skills, experience/knowledge, and education employers are most often looking for in data scientist candidates, but you cannot look through hundreds or thousands of jobs to figure this out, and looking through a small sample might give you the impression that a certain skill/tool/degree is more desireable than it really is. 

Instead, I'll check the job descriptions in my data set for mentions of various data science skills, tools, and degrees, and record the results for each one within the data science jobs DataFrame (`dsjobs`). This feature engineering and analysis of the results will allow me to identify the most desireable traits employers are looking for in candidates, and which skills/experiences tend to be mentioned together (possibly indicating a relationship). 

### Skills: search for mentions of data science skills

What the role of a data scientist is, and which skills they should have, will vary depending on who you ask, because:
- data scientists working in different sectors and industries will have different roles
- the label "data scientist" has now also been applied to jobs that were previously called something else, e.g. statisticians, data analysts, computer scientists, and 
- there is a growing demand for specialists with certain combinations of data science skills

 To learn which skills are most commonly mentioned (and found together) in data science job advertisements, I will create and use a dictionary of regular expressions to search for a wide range of skills within the job description text, and add the results to `dsjobs`. 
 
 The data science skills I have decided to look for are based on my own knowledge of the data scientist role, scanning many of the job descriptions in the data set, and additional web searches.

In [7]:
# The function below will be used to develop and optimise the regular expressions 
# for searching the the job descriptions text for mentions of skills, tools, etc

def check_re(jobsdf, regex):
    """ Scans through a random sample of 100 job descriptions from the jobs DataFrame, /
    looking for a match to the regular expression(s), and prints up to 20 match objects /
    plus the surrounding text (for context)

    :param jobsdf: jobs data with job descriptions
    :type jobsdf: pandas.core.frame.DataFrame
    :param regex: regular expression(s)
    :type regex: str | dict
    
    """    
    print(f"*** Pattern = '{regex}' ***\n")

    # produce a random integer for the sample function
    seed = random.randint(0, 100)

    if (type(regex) is str):
        sample_counter = 1
        print_counter = 0
        for i in (jobsdf.job_description.sample(n=100, random_state=seed)):
            if print_counter >= 20:
                break
            result = re.search(regex, i, re.IGNORECASE)  # returns 1st occurrence only
            if result:
                start = result.span()[0]
                stop = result.span()[1]
                string = i[start-50:stop+50]
                print(f"[{sample_counter}] {string}\n")
                print_counter += 1
            sample_counter += 1
    else:
        raise TypeError(
            "regex must be a string"
        )


In [8]:
# create a dictionary of regular expression that will be used to search for 
# data science skills in each job description; the labels will later be used for plots
dict_skills = {
    "shell_or_bash": {
        # placing a 'r' before a string literal creates a raw-string literal
        # raw strings do not process escape sequences (\n, \b, etc.) 
        're': r"\bshell\b|\bbash\b",  
        'label': "Shell/Bash"
    },
    "version_control": {
        're': "version control",
        'label': "Version Control"
    },
    "distributed_data": {
        're': "distributed data",
        'label': "Distributed Data"
    },
    "big_unstructured_data": {
        're': "big data|NoSQL|data lake|unstructured data",
        'label': "Big Data/Unstructured Data/NoSQL"
    },
    "data_management": {
        're': r"data\w* management",
        'label': "Data Management"
    },
    "data_transformation": {
        're': r"data transform\w*|\betl\b",
        'label': "Data Transformation"
    },
    "data_mining": {
        're': r"\bmine\b|\bmining\b",
        'label': "Data Mining"
    },
    "data_streaming": {
        're': r"\bdata stream\w*",
        'label': "Data Streaming"
    },
    "data_analytics": {
        're': r"data analy|analy[sz]e data\w*",
        'label': "Data Analytics"
    },
    "querying": {
        're': r"\bquer\w*",
        'label': "Querying"
    },
    "data_cleaning": {
        're': r"\bcleaning\w*|\bwrangl\w*|\bmung\w*",
        'label': "Data Cleaning"
    },
    "geospatial": {
        're': "geospatial",
        'label': "Geospatial Data"
    },
    "time_series": {
        're': "time series|time-series",
        'label': "Time Series Data"
    },
    "web_scraping": {
        're': "web scraping|scrape",
        'label': "Web Scraping"
    },
    "data_visualisation": {
        're': r"data visuali\w*",
        'label': "Data Visualisation"
    },
    "dashboard": {
        're': "dashboard",
        'label': "Building Dashboards"
    },
    "statistical_modelling": {
        're': r"predictive analy\w*|predictive model\w*|statistical model\w*",
        'label': "Statistical Modelling"
    },
    "regression": {
        're': "regression",
        'label': "Regression"
    },
    "clustering": {
        're': "cluster|clustering",
        'label': "Clustering"
    },
    "classification": {
        're': r"classif\w*|decision tree|random forest|svm|support vector machine",
        'label': "Classification"
    },
    "supervised_learning": {
        're': r"\bsupervised",
        'label': "Supervised Learning"
    },
    "unsupervised_learning": {
        're': "unsupervised",
        'label': "Unsupervised Learning"
    },
    "machine_learning": {
        're': r"machine learning|\bml\b",
        'label': "Machine Learning"
    },
    "deep_learning": {
        're': "deep learning|neural network",
        'label': "Deep Learning"
    },
    "dimensionality_reduction": {
        're': r"dimensionality reduction|principle component analysis|PCA|t-SNE",
        'label': "Dimensionality Reduction"
    },
    "sequence_modelling": {
        're': r"sequence model\w*|rnn",
        'label': "Sequence Modelling"
    },
    "computer_vision": {
        're': r"computer vision|machine vision|image classif\w*",
        'label': "Computer Vision"
    },
    "NLP": {
        're': "natural language processing|NLP",
        'label': "Natural Language Processing"
    },
    "ai": {
        're': r"artificial intelligence|\bai\b",
        'label': "Artifical Intelligence"
    },
    "hypothesis_testing": {
        're': r"\ba\/b\b|hypothesis testing",
        'label': "Hypothesis Testing"
    },
    "cloud_computing": {
        're': "cloud|cloud computing",
        'label': "Cloud Computing"
    },
    "software_development": {
        're': "software development|software engineering",
        'label': "Software Development/Engineering"
    },
    "agile_working": {
        're': "agile working|agile develop|agile method|scrum",
        'label': "Agile Working"
    },
    "pipelines": {
        're': "pipeline",
        'label': "Working with/Developing Pipelines"
    },
    "devops": {
        're': r"\bci\b|\bcd\b|devops",
        'label': "DevOps"
    },
    "deployment": {
        're': r"deploy\w*",
        'label': "Deploying Models/Products"
    },
    "containerization": {
        're': r"container\w*",
        'label': "Containerisation"
    },
}


In [ ]:
# use check_re to test the regex dictionary that will be used to search for skills
for key, val in dict_skills.items():
    print(f"{key.upper()}\n")
    check_re(jobsdf=dsjobs, regex=val["re"])

In [10]:
# lambda function below throws a SettingWithCopyWarning but I have checked that
# I am not making a change to a copy so this warning will be suppressed
from warnings import simplefilter
from pandas.core.common import SettingWithCopyWarning
simplefilter(action="ignore", category=SettingWithCopyWarning)

In [11]:
# for each skill in the dict_skills, create a column in the dataframe with the 'label',
# with a lambda function, search for the skill in each job's description text using 
# the regular expression ('re'), entering the boolean result in the column, 'label'
for key, val in dict_skills.items():
    dsjobs[key] = dsjobs["job_description"].apply(
        lambda x: bool(re.search(val['re'], x, re.IGNORECASE)))
    # # if you want to see the results for debugging, uncomment below
    # print(f"{val['label']} : {round(np.mean(dsjobs[key])*100, 2)}%")


### Tools: search for mentions of data science tools

Job descriptions will almost always name programming languages and other data science tools that the company uses or the employer expects candidates to know/have experience with. I want to know which tools are most popular among employers/companies, so I will create and test a dictionary of regular expressions to search for a wide range of tools within the description text of each job, and add the results to `dsjobs`. The data science tools I have decided to look for are based on my own knowledge of the data scientist role, scanning many of the job descriptions in the data set, and additional web searches.

In [12]:
# create a dictionary of regular expression that will be used to search for 
# data science tools in each job description; the labels will later be used for plots
dict_tools = {
    "python": {
        # programming language
        're': r"\bpython\b",
        'label': "Python"
    },
    "r": {
        # programming language
        're': r"\br(?!&)\b|\brstudio\b",
        'label': "R"
    },
    "scala": {
        # programming language
        're': r"\bscala\b",
        'label': "Scala"
    },
    "matlab": {
        # desktop environment tuned for iterative analysis and design processes with a 
        # programming language that expresses matrix and array mathematics directly
        're': r"\bmatlab\b",
        'label': "MATLAB"
    },
    "sas": {
        # statistical software suite
        're': r"\bsas\b",
        'label': "SAS"
    },
    "spss": {
        # software package used for interactive, or batched, statistical analysis
        're': r"\bspss\b",
        'label': "SPSS"
    },
    "stata": {
        # statistical software package
        're': r"\bstata\b",
        'label': "Stata"
    },
    "perl": {
        # programming language
        're': r"\bperl\b",
        'label': "Perl"
    },
    "unix": {
        # Operating system
        're': r"\bunix\b",
        'label': "Unix"
    },
    "linux": {
        # Unix-like operating system
        're': r"\blinux\b",
        'label': "Linux"
    },
    "git_github": {
        # Git is a version control system that lets you manage and keep track of your source code history;
        # GitHub is a cloud-based hosting service that lets you manage Git repositories.
        're': r"\bgit|github\b",
        'label': "Git/GitHub"
    },
    "anaconda": {
        # Anaconda is a distribution of the Python and R programming languages for scientific computing
        # (data science, machine learning applications, large-scale data processing, predictive analytics, etc.),
        # that aims to simplify package management and deployment.
        're': r"\banaconda\b",
        'label': "Anaconda"
    },
    "d3js": {
        # a JavaScript library for visualizing data with HTML, SVG, and CSS
        're': r"\bd3\.?js\b",
        'label': "D3.js"
    },
    "jira": {
        # software used for bug tracking, issue tracking, and project management
        're': r"\bjira\b",
        'label': "Jira"
    },
    "java": {
        # programming language
        're': r"\bjava\b",
        'label': "Java"
    },
    "javascript": {
        # programming language
        're': r"\bjavascript\b",
        'label': "JavaScript"
    },
    "nodejs": {
        # cross-platform, back-end JavaScript runtime environment that executes JavaScript code outside a web browser.
        're': r"\bnode\.js\b",
        'label': "Node.js"
    },
    "c++_c#": {
        # programming languages
        're': r"\bc\b|\bc\+{2}\b|\bc\#\b",
        'label': "C++/C#"
    },
    "docker": {
        # Docker is a set of products that use OS-level virtualization to deliver software in packages called containers,
        # removing the issue of dependencies
        're': r"\bdocker\w*?\b",
        'label': "Docker"
    },
    "kubernetes": {
        # Container-orchestration system for automating computer application deployment, scaling, and management
        're': r"\bkubernetes\b",
        'label': "Kubernetes"
    },
    "google_cloud": {
        # a suite of cloud computing services
        're': r"\bgoogle cloud\b|\bgcp\b",
        'label': "Google Cloud Platform (GCP)"
    },
    "aws": {
        # a suite of cloud computing services
        're': r"\baws\b",
        'label': "Amazon Web Services (AWS)"
    },
    "azure": {
        # a suite of cloud computing services
        're': r"\bazure\b",
        'label': "Microsoft Azure"
    },
    "kafka": {
        # a stream-processing software platform for handling real-time data feeds, which can connect to external systems (for data import/export)
        're': r"\bkafka\b",
        'label': "Kafka"
    },
    "kinesis": {
        # a cloud-based service that allows real-time processing of data streaming large amount of data
        're': r"\bkinesis\b",
        'label': "Kinesis"
    },
    "spark": {
        # general-purpose cluster-computing framework
        're': r"\bspark\b",
        'label': "Spark"
    },
    "elasticsearch": {
        # Elasticsearch takes unstructured data from different locations, stores and indexes it according to user-specified mapping
        # (or automatically from data) and makes it searchable.
        're': r"\belasticsearch\b",
        'label': "Elasticsearch"
    },
    "sql": {
        # a domain-specific language used in programming and designed for managing data held in a relational database management system
        're': r"\bsql\b",
        'label': "Structured Query Language (SQL)"
    },
    "redshift": {
        #  a cloud-based, big data warehouse product; can be used for real-time analytics
        're': r"\bredshift\b",
        'label': "Redshift"
    },
    "looker": {
        # a browser-based data analytics platform for collection, visualization and analysis
        're': r"\blooker\b",
        'label': "Looker"
    },
    "bigquery": {
        # A serverless cloud storage platform for large data sets,
        # which allows you to run complex analytical SQL-based queries under large sets of data
        're': r"\bbigquery\b",
        'label': "BigQuery"
    },
    "hive": {
        # A data warehouse software that allows users to read, write, and manage petabytes of data using SQL.
        # Hive is built on top of Hadoop
        're': r"\bhive\b",
        'label': "Apache Hive"
    },
    "excel": {
        # A program that allows users to organize, format and calculate data with formulas using a spreadsheet system
        're': r"\bexcel\b",
        'label': "Excel"
    },
    "power_bi": {
        # Power BI is a business analytics service by Microsoft for interactive visualizations and business intelligence
        # capabilities with an interface simple enough for end users to create their own reports and dashboards.
        're': r"\bpower bi\b",
        'label': "Power BI"
    },
    "snowflake": {
        # Snowflake is a full SQL data warehouse built for the cloud (on top of the AWS or Azure)
        're': r"\bsnowflake\b",
        'label': "Snowflake"
    },
    "tableau": {
        # an interactive data visualization software
        're': r"\btableau\b",
        'label': "Tableau"
    },
    "vertica": {
        # a data analytics platform
        're': r"\bvertica\b",
        'label': "Vertica"
    },
    "grafana": {
        # a multi-platform open source analytics and interactive visualization web application.
        're': r"\bgrafana\b",
        'label': "Grafana"
    },
    "lubridate": {
        # R library used for data wrangling, good for date-time data
        're': r"\blubridate\b",
        'label': "Lubridate"
    },
    "tidyverse": {
        # R data science packages
        're': r"\btidyverse\b",
        'label': "Tidyverse"
    },
    "dplyr": {
        # R package to manipulate, clean and summarize unstructured data
        're': r"\bdplyr\b",
        'label': "dplyr"
    },
    "ggplot2": {
        # library for data visualization in R
        're': r"\bggplot2\b",
        'label': "ggplot2"
    },
    "esquisse": {
        # R ggplot2 addin that allows you to interactively explore your data by visualizing it with the ggplot2 package,
        # then export the graph or retrieve the code generating the graph
        're': r"\besquisse\b",
        'label': "esquisse"
    },
    "shiny": {
        # R package  to build interactive web apps
        're': r"\bshiny\b",
        'label': "Shiny"
    },
    "bioconductor": {
        # software for bioinformatics
        're': r"\bbioconductor\b",
        'label': "Bioconductor"
    },
    "knitr": {
        # R package for dynamic report generation
        're': r"\bknitr\b",
        'label': "knitr"
    },
    "rmarkdown": {
        # R package for dynamic report generation
        're': r"\brmarkdown\b",
        'label': "RMarkdown"
    },
    "quanteda": {
        # R package for managing and analyzing text
        're': r"\bquanteda\b",
        'label': "quanteda"
    },
    "rcrawler": {
        # R package for domain-based web crawling and content scraping
        're': r"\brcrawler\b",
        'label': "RCrawler"
    },
    "caret": {
        # R package for model building and evaluation, e.g. data splitting, pre-processing, feature selection, variable importance estimation etc.
        're': r"\bcaret\b",
        'label': "Caret"
    },
    "mlr": {
        # R package and framework for ML
        're': r"\bmlr\b",
        'label': "mlr"
    },
    "oracle": {
        # cloud computing infrastructure
        're': r"\boracle\b",
        'label': "Oracle"
    },
    "numpy": {
        # python library adding support for large, multi-dimensional arrays and matrices,
        # along with a large collection of high-level mathematical functions to operate on them
        're': r"\bnumpy\b",
        'label': "NumPy"
    },
    "scipy": {
        # python library used for scientific computing and technical computing, with modules for optimization,
        # linear algebra, integration, interpolation, special functions, FFT, signal and image processing,
        # ODE solvers and other tasks common in science and engineering.
        're': r"\bscipy\b",
        'label': "SciPy"
    },
    "pandas": {
        # python library for data manipulation and analysis in python, offering data structures and operations
        # for manipulating numerical tables and time series.
        're': r"\bpandas\b",
        'label': "pandas"
    },
    "matplotlib": {
        # python ibrary for creating static, animated, and interactive visualizations in Python, based on NumPy
        're': r"\bmatplotlib\b",
        'label': "matplotlib"
    },
    "bokeh": {
        # python library used to make interactive plots, dashboards, and data applications for modern web browsers/notebooks
        're': r"\bbokeh\b",
        'label': "Bokeh"
    },
    "sklearn": {
        # python ML library, used for classification, regression clustering, SVM, random forests,
        # gradient boosting, k-means and DBSCAN; designed to work with NumPy and SciPy.
        're': r"\bscikit\-learn\b|\bsklearn\b",
        'label': "scikit-learn"
    },
    "pytorch": {
        # python ML library based on the Torch library, uses include computer vision and NLP
        're': r"\bpytorch\b",
        'label': "PyTorch"
    },
    "pyspark": {
        # python API written to support Apache Spark
        're': r"\bpyspark\b",
        'label': "PySpark"
    },
    "sparkml": {
        # Spark MLlib is a distributed ML framework on top of Spark Core
        're': r"\bspark[\.\s]?ml",
        'label': "Spark ML"
    },
    "tensorflow": {
        # library for ML with a particular focus on training and interference of deep neural networks
        're': r"\btensorflow\b",
        'label': "TensorFlow"
    },
    "spacy": {
        # Library for advanced NLP
        're': r"\bspacy\b",
        'label': "spaCy"
    },
    "keras": {
        # Library providing Python interface for artificial neural networks.
        're': r"\bkeras\b",
        'label': "Keras"
    },
    "databricks": {
        # Data analytics/ data lake platform optimized for Azure
        're': r"\bdatabricks\b",
        'label': "Databricks"
    },
    "mongodb": {
        # Cross-platform document-oriented NoSQL database program.
        're': r"\bmongodb\b",
        'label': "MongoDB"
    },
    "hadoop": {
        # A software framework for distributed storage and processing of big data
        're': r"\bhadoop\b",
        'label': "Hadoop"
    },
}


In [ ]:
# use check_re to test the regular expressions that will be used to search for tools
for key, val in dict_tools.items():
    print(f"{key.upper()}\n")
    check_re(jobsdf=dsjobs, regex=val["re"])

The regular expression looking for mentions of the C, C++ or C# programming languages matches mentions of "C" with respect to grades, e.g. GCSE grades or NHS/other salary bands, so I will check for this and correct the results where needed.

In [14]:
# check (and correct, if needed) the results of the regex used to identify mentions of 
# C, C++ or C# programming languages
for index, row in dsjobs.iterrows():
    # return an iterator over all matches with the regular expression
    result = re.finditer(
        dict_tools["c++_c#"]['re'], row["job_description"], re.IGNORECASE)
    if result:  # if there are any matches:
        for v in result:
            start = v.span()[0]
            stop = v.span()[1]
            # isolate the text surrounding the match result (string)
            string = row['job_description'][start-50:stop+50]
            counter = 0
            # search this string for reference to grades, GCSEs and Bands (e.g. NHS salary bands)
            if re.search(r"\bgcse\b|\bband\b|\bgrade\b", string, re.IGNORECASE):
                # correct these "false positives"
                print(string+"\n")
                dsjobs.loc[index, "c++_c#"] = False
                


In [15]:
# search each job description for each skill using the regular expressions in dict_tools,
# entering the boolean result in a new column for each tool
for key, val in dict_tools.items():
    dsjobs[key] = dsjobs["job_description"].apply(
        lambda x: bool(re.search(val['re'], x, re.IGNORECASE)))
    # print(f"{val['label']}: {(np.mean(dsjobs[key])*100).round(2)}%")


### Education: search for mentions of academic qualifications

 A quick scan of a few data science roles will tell you that employers believe a very strong educational background is required to develop the depth of knowledge necessary to be a data scientist.
 
 To find out what proportion of data science jobs require a degree, and which qualifications and subjects are most often mentioned by employers in job descriptions, I will create dictionaries of regular expressions to search for these within the job descriptions, and add the results (boolean masks) to `dsjobs`.

In [16]:
# create a dictionary of regular expressions and labels for degree qualifications,
# which will be used to search for these in the job descriptions
# the labels will later be used for plots
dict_qualifications = {
    "bachelors": {
        're': r"\bbachelor\'?|\bhonours\b|\bbsc\b|\bbs\b|\bba\b",
        'label': "Bachelor's"
    },
    "masters": {
        're': r"\bmaster\'?s\b|\bmsc\b|\bma\b",
        'label': "Master's"
    },
    "phd": {
        're': r"\bphd\b|\bdphil\b|\bdoctorate\b",
        'label': "PhD"
    },
}

In [ ]:
# use check_re to test the regular expressions that will be used for searching qualifications
for key, val in dict_qualifications.items():
    print(f"{key.upper()}\n")
    check_re(jobsdf=dsjobs, regex=val["re"])

In [18]:
# search each job description for each qualification using the regular expressions in 
# dict_qualifications, entering the boolean result in a new column for each one
for key, val in dict_qualifications.items():
    dsjobs[key] = dsjobs["job_description"].apply(
        lambda x: bool(re.search(val['re'], x, re.IGNORECASE)))
    # for debugging: if any job mentions a qualification, column mean should be non-zero
    print(f"{val['label']}: {(np.mean(dsjobs[key])).round(2)}")


Bachelor's: 0.19
Master's: 0.24
PhD: 0.28


To find mentions of degree subjects in the job descriptions, searching for just the name of the subject in the job description will not do, since words like "statistics" and "engineering" are also mentioned in other contexts, e.g. in the company overview, or in the description of responsibilities, so I'll look for instances where the subject name and reference to a degree are in the same sentence.  

In [19]:
# split the job description text into sentences using new lines
dsjobs['job_description_sent'] = dsjobs["job_description"].apply(
    lambda x: x.split("\n")
)

In [20]:
# create a dictionary of regular expressions for degree subjects, which will
# be used to search for the subjects in the text surrounding the mention of 
# a qualification; the labels will be used for analysis plots later
dict_degsubjects = {
    "degree_statistics": {
        're': r"\bstatistics\b",
        'label': "Statistics"
    },
    "degree_compsci": {
        're': r"\bcomputer science\b",
        'label': "Computer Science"
    },
    "degree_software_engineering": {
        're': r"\bsoftware engineering\b",
        'label': "Software Engineering"
    },
    "degree_economics": {
        're': r"\beconomics\b",
        'label': "Economics"
    },
    "degree_maths": {
        're': r"\bmaths\b|\bmathematics\b",
        'label': "Maths"
    },
    "degree_physics": {
        're': r"\bphysics\b",
        'label': "Physics"
    },
    "degree_engineering": {
        're': r"\bengineering\b",
        'label': "Engineering"
    },
    "degree_biology": {
        're': r"\bbiolog\b|\bbioinformatics\b",
        'label': "Biology"
    },
    "degree_chemistry": {
        're': r"\bchemistry\b",
        'label': "Chemistry"
    },
    "degree_datasci": {
        're': r"\bdata science\b",
        'label': "Data Science"
    },
    "degree_ml_ai": {
        're': r"machine learning|\bml\b|artificial intelligence|\bai\b",
        'label': "ML or AI"
    },
    "degree_stem_quantfield": {
        're': r"\bstem\b|\bquantitative field\b",
        'label': "'STEM' or 'Quantitative Field'"
    },
}


In [21]:
# line by line, check each job description for the subject + mention of a 'degree'/qualification
for key, val in dict_degsubjects.items():
    # create a column for the degree subject in ds with default values set to False
    dsjobs[key] = False
    for index, row in dsjobs.iterrows():
        for sentence in row['job_description_sent']:
            # if the subject is in the sentence
            if (re.search(val['re'], sentence, re.IGNORECASE)):
                # if "degree" is in the same sentence, positive result
                if (re.search(r"degree", sentence, re.IGNORECASE)):
                    dsjobs.loc[index, key] = True
                    break   # no need to look for a specific degree type
                # else look for a specific qualification, e.g. "PhD"
                else:
                    for k, v in dict_qualifications.items():
                        # if the qualification is in the same sentence
                        if (re.search(v['re'], sentence, re.IGNORECASE)):  
                            dsjobs.loc[index, key] = True
                            break   # no need to keep looking for a qualification
    # for debugging: if any mentions of a degree in the subject, column mean should be non-zero
    print(f"{val['label']}: {round((dsjobs[key].mean()),2)}")


Statistics: 0.26
Computer Science: 0.29
Software Engineering: 0.01
Economics: 0.06
Maths: 0.29
Physics: 0.12
Engineering: 0.17
Biology: 0.02
Chemistry: 0.02
Data Science: 0.13
ML or AI: 0.1
'STEM' or 'Quantitative Field': 0.11


In [22]:
# create a boolean mask to mark jobs that mention any type of degree
degree_any = dsjobs.loc[:,"degree_statistics":"degree_stem_quantfield"].any(axis='columns')
dsjobs["degree_any"] = degree_any

# add mention of "any degree" to the qualifcations dictionary for analysis and visualisation
dict_qualifications["degree_any"] = {'label': "Any Degree"}


In [23]:
# create a dictionary of the dictionaries so we can use them later
dsjobs_dicts = {
    'dict_skills' : dict_skills,
    'dict_tools' : dict_tools,
    'dict_degsubjects' : dict_degsubjects,
    'dict_qualifications' : dict_qualifications,
}

# save the list of dictionaries as a .pkl file
with open('dsjobs_dicts.pkl', 'wb') as f:
    pickle.dump(dsjobs_dicts, f)

In [24]:
# save the fully wrangled dsjobs dataframe 
# as a .csv file
dsjobs.to_csv(
    os.path.join(path, f'dsjobs_df_{scrapedate}_wrangled.csv'), 
    encoding='utf-8'
)
# as a .pkl file which preserves data types (better for processing steps)
dsjobs.to_pickle(os.path.join(path, f'dsjobs_df_{scrapedate}_wrangled.pkl'))
